<a href="https://colab.research.google.com/github/DonghakPark/Dacon_Competition/blob/master/08_24(colab).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from time import time
import datetime
import gc
from scipy.signal import find_peaks, peak_widths, peak_prominences
# Model
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder

import lightgbm as lgb
import xgboost as xgb

# Ensemble
from scipy.optimize import minimize
from sklearn.metrics import log_loss

In [ ]:
train = pd.read_csv('')